# House Prices - Advanced Regression Techniques
*Predict sales prices and practice feature engineering, RFs, and gradient boosting*


## Data Fields

*   **SalePrice**: the property's sale price in dollars. This is the target variable that you're trying to predict.
*   **MSSubClass**: The building class
*   **MSZoning**: The general zoning classification
*   **LotFrontage**: Linear feet of street connected to property
*   **LotArea**: Lot size in square feet
*   **Street**: Type of road access
*   **Alley**: Type of alley access
*   **LotShape**: General shape of property
*   **LandContour**: Flatness of the property
*   **Utilities**: Type of utilities available
*   **LotConfig**: Lot configuration
*   **LandSlope**: Slope of property
*   **Neighborhood**: Physical locations within Ames city limits
*   **Condition1**: Proximity to main road or railroad
*   **Condition2**: Proximity to main road or railroad (if a second is present)
*   **BldgType**: Type of dwelling
*   **HouseStyle**: Style of dwelling
*   **OverallQual**: Overall material and finish quality
*   **OverallCond**: Overall condition rating
*   **YearBuilt**: Original construction date
*   **YearRemodAdd**: Remodel date
*   **RoofStyle**: Type of roof
*   **RoofMatl**: Roof material
*   **Exterior1st**: Exterior covering on house
*   **Exterior2nd**: Exterior covering on house (if more than one material)
*   **MasVnrType**: Masonry veneer type
*   **MasVnrArea**: Masonry veneer area in square feet
*   **ExterQual**: Exterior material quality
*   **ExterCond**: Present condition of the material on the exterior
*   **Foundation**: Type of foundation
*   **BsmtQual**: Height of the basement
*   **BsmtCond**: General condition of the basement
*   **BsmtExposure**: Walkout or garden level basement walls
*   **BsmtFinType1**: Quality of basement finished area
*   **BsmtFinSF1**: Type 1 finished square feet
*   **BsmtFinType2**: Quality of second finished area (if present)
*   **BsmtFinSF2**: Type 2 finished square feet
*   **BsmtUnfSF**: Unfinished square feet of basement area
*   **TotalBsmtSF**: Total square feet of basement area
*   **Heating**: Type of heating
*   **HeatingQC**: Heating quality and condition
*   **CentralAir**: Central air conditioning
*   **Electrical**: Electrical system
*   **1stFlrSF**: First Floor square feet
*   **2ndFlrSF**: Second floor square feet
*   **LowQualFinSF**: Low quality finished square feet (all floors)
*   **GrLivArea**: Above grade (ground) living area square feet
*   **BsmtFullBath**: Basement full bathrooms
*   **BsmtHalfBath**: Basement half bathrooms
*   **FullBath**: Full bathrooms above grade
*   **HalfBath**: Half baths above grade
*   **Bedroom**: Number of bedrooms above basement level
*   **Kitchen**: Number of kitchens
*   **KitchenQual**: Kitchen quality
*   **TotRmsAbvGrd**: Total rooms above grade (does not include bathrooms)
*   **Functional**: Home functionality rating
*   **Fireplaces**: Number of fireplaces
*   **FireplaceQu**: Fireplace quality
*   **GarageType**: Garage location
*   **GarageYrBlt**: Year garage was built
*   **GarageFinish**: Interior finish of the garage
*   **GarageCars**: Size of garage in car capacity
*   **GarageArea**: Size of garage in square feet
*   **GarageQual**: Garage quality
*   **GarageCond**: Garage condition
*   **PavedDrive**: Paved driveway
*   **WoodDeckSF**: Wood deck area in square feet
*   **OpenPorchSF**: Open porch area in square feet
*   **EnclosedPorch**: Enclosed porch area in square feet
*   **3SsnPorch**: Three season porch area in square feet
*   **ScreenPorch**: Screen porch area in square feet
*   **PoolArea**: Pool area in square feet
*   **PoolQC**: Pool quality
*   **Fence**: Fence quality
*   **MiscFeature**: Miscellaneous feature not covered in other categories
*   **MiscVal**: $Value of miscellaneous feature
*   **MoSold**: Month Sold
*   **YrSold**: Year Sold
*   **SaleType**: Type of sale
*   **SaleCondition**: Condition of sale

## Data-Processing

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

### Load Datasets

In [ ]:
train_data = pd.read_csv('https://raw.githubusercontent.com/keivanipchihagh/Kaggle-Competitions/main/House%20Prices%20-%20Advanced%20Regression%20Techniques/Datasets/Raw%20Datasets/train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/keivanipchihagh/Kaggle-Competitions/main/House%20Prices%20-%20Advanced%20Regression%20Techniques/Datasets/Raw%20Datasets/test.csv')

train_data.head(10)

### Handle Missing Data

In [5]:
# null_count = train_data.isnull().sum()
# null_percentage = null_count / train_data.count()
# pd.DataFrame(columns = ['Column Name', 'Null Entries', 'Total Entries', 'Null Percentage'], data = train_data.isnull().sum())
train_data.count()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64